In [10]:
# @title 🚀 Enhanced Study Helper Chatbot - Run this cell first!
import json
import random
import re
from IPython.display import display, Markdown
import ipywidgets as widgets
from datetime import datetime

# ====== ENHANCED TOPICS DATABASE ======
topics_data = {
    "topics": [
        # Biology
        {
            "name": "Photosynthesis",
            "aliases": ["plant energy", "light reaction"],
            "explanation": "Photosynthesis is the process by which green plants use sunlight to synthesize foods with chlorophyll.",
            "formula": "6CO₂ + 6H₂O + light → C₆H₁₂O₆ + 6O₂",
            "examples": ["Leaves turning sunlight into energy", "Algae producing oxygen in water"],
            "related": ["Chlorophyll", "Respiration", "Carbon Cycle"],
            "quiz": [
                {
                    "type": "short",
                    "question": "What are the main products of photosynthesis?",
                    "answer": "Glucose and oxygen"
                },
                {
                    "type": "mcq",
                    "question": "Which organelle performs photosynthesis?",
                    "options": ["Mitochondria", "Chloroplast", "Nucleus", "Ribosome"],
                    "answer": "Chloroplast"
                }
            ]
        },

        # Computer Science
        {
            "name": "Python Programming",
            "aliases": ["python", "coding"],
            "explanation": "Python is a high-level, interpreted programming language known for its readability and versatility.",
            "examples": [
                "Web development (Django, Flask)",
                "Data science (Pandas, NumPy)",
                "Machine learning (TensorFlow, PyTorch)"
            ],
            "related": ["Variables", "Functions", "OOP", "Algorithms"],
            "quiz": [
                {
                    "type": "short",
                    "question": "How do you create a list in Python?",
                    "answer": "Using square brackets: my_list = [1, 2, 3]"
                },
                {
                    "type": "mcq",
                    "question": "Which keyword defines a function in Python?",
                    "options": ["def", "function", "fun", "define"],
                    "answer": "def"
                }
            ]
        },
        {
            "name": "Artificial Intelligence",
            "aliases": ["AI", "machine learning"],
            "explanation": "AI involves creating computer systems that can perform tasks requiring human intelligence.",
            "categories": ["Machine Learning", "Neural Networks", "Computer Vision", "NLP"],
            "examples": ["Chatbots", "Image recognition", "Self-driving cars"],
            "related": ["Deep Learning", "Data Science", "Robotics"],
            "quiz": [
                {
                    "type": "mcq",
                    "question": "Which is NOT a type of machine learning?",
                    "options": ["Supervised", "Unsupervised", "Reinforcement", "Guessing"],
                    "answer": "Guessing"
                }
            ]
        },
        {
            "name": "Computer Networks",
            "aliases": ["networking", "internet protocols"],
            "explanation": "Computer networks connect devices to share resources and information.",
            "key_concepts": ["TCP/IP", "DNS", "HTTP/HTTPS", "LAN/WAN"],
            "related": ["Cybersecurity", "Cloud Computing", "Web Development"],
            "quiz": [
                {
                    "type": "short",
                    "question": "What does DNS stand for?",
                    "answer": "Domain Name System"
                }
            ]
        },

        # Math
        {
            "name": "Algebra",
            "aliases": ["linear equations", "polynomials"],
            "explanation": "Algebra is mathematics with variables and equations to represent relationships.",
            "formula": "ax² + bx + c = 0",
            "examples": ["Solving for x: 2x + 5 = 15", "Quadratic equations"],
            "related": ["Geometry", "Calculus", "Trigonometry"],
            "quiz": [
                {
                    "type": "short",
                    "question": "What is the quadratic formula?",
                    "answer": "x = [-b ± √(b²-4ac)] / 2a"
                }
            ]
        }
    ],
    "study_tips": [
        "Use the Pomodoro technique (25 min study, 5 min break)",
        "Create mind maps for complex topics",
        "Teach concepts to someone else to reinforce learning",
        "Practice with flashcards for memorization",
        "Solve problems actively instead of just reading"
    ],
    "definitions": {
        "algorithm": "A step-by-step procedure for solving a problem",
        "variable": "A named storage location in programming",
        "function": "A reusable block of code that performs a specific task",
        "oop": "Object-Oriented Programming: organizing code as objects with properties and methods"
    }
}

# ====== ENHANCED CHATBOT CLASS ======
class StudyBot:
    def __init__(self):
        self.topics = topics_data
        self.quiz_active = False
        self.current_quiz = []
        self.quiz_score = 0
        self.current_question_index = 0
        self.quiz_topic = ""
        self.start_time = None

    def _get_topic_info(self, topic: str) -> dict:
        """Find topic in database with fuzzy matching"""
        topic_lower = topic.lower()
        for t in self.topics["topics"]:
            if (topic_lower in t["name"].lower() or
                any(topic_lower in alt.lower() for alt in t.get("aliases", []))):
                return t
        return None

    def explain_topic(self, topic: str) -> str:
        """Generate detailed explanation of a topic"""
        topic_data = self._get_topic_info(topic)
        if not topic_data:
            return f"❌ I don't know about '{topic}'. Try: Python, AI, or Photosynthesis."

        response = f"📚 **{topic_data['name']}**\n\n"
        response += f"{topic_data['explanation']}\n\n"

        if "formula" in topic_data:
            response += f"🔢 Formula: {topic_data['formula']}\n\n"

        if "examples" in topic_data:
            response += "💡 Examples:\n"
            response += "\n".join(f"- {ex}" for ex in topic_data["examples"]) + "\n\n"

        if "key_concepts" in topic_data:
            response += "🔑 Key Concepts:\n"
            response += ", ".join(topic_data["key_concepts"]) + "\n\n"

        if "related" in topic_data:
            response += f"🔗 Related Topics: {', '.join(topic_data['related'])}"

        return response

    def get_definition(self, term: str) -> str:
        """Get definition of a technical term"""
        term_lower = term.lower()
        for word, definition in self.topics["definitions"].items():
            if term_lower == word.lower():
                return f"📖 **{word.capitalize()}**: {definition}"
        return f"❌ I don't have a definition for '{term}'. Try: algorithm, variable, or OOP."

    def get_study_tip(self) -> str:
        """Return a random study tip"""
        return "📌 **Study Tip**: " + random.choice(self.topics["study_tips"])

    def start_quiz(self, topic: str = None) -> str:
        """Initialize a quiz session"""
        self.quiz_active = True
        self.quiz_score = 0
        self.current_question_index = 0
        self.start_time = datetime.now()

        if topic:
            topic_data = self._get_topic_info(topic)
            if topic_data and "quiz" in topic_data:
                self.current_quiz = topic_data["quiz"]
                self.quiz_topic = topic_data["name"]
            else:
                self.quiz_active = False
                return f"❌ No quiz available for '{topic}'. Try: Python, AI, or Algebra."
        else:
            # General quiz with questions from all topics
            self.current_quiz = []
            for t in self.topics["topics"]:
                if "quiz" in t:
                    self.current_quiz.extend(t["quiz"])
            self.quiz_topic = "General Knowledge"
            if not self.current_quiz:
                self.quiz_active = False
                return "❌ No quiz questions available."

        random.shuffle(self.current_quiz)
        return self._format_question(0)

    def _format_question(self, index: int) -> str:
        """Format the current question with options if MCQ"""
        if index >= len(self.current_quiz):
            return self.end_quiz()

        question = self.current_quiz[index]
        formatted = f"📝 **Question {index+1}/{len(self.current_quiz)}** ({self.quiz_topic})\n\n"
        formatted += f"{question['question']}\n"

        if question["type"] == "mcq":
            for i, option in enumerate(question["options"], 1):
                formatted += f"{i}. {option}\n"
            formatted += "\n(Type the number or full answer)"

        return formatted

    def check_answer(self, user_answer: str) -> str:
        """Check user's answer and provide feedback"""
        if not self.quiz_active or self.current_question_index >= len(self.current_quiz):
            return "No active quiz question."

        question = self.current_quiz[self.current_question_index]
        correct_answer = question["answer"].lower()
        user_answer = user_answer.strip().lower()

        # Handle MCQ numeric answers
        if question["type"] == "mcq" and user_answer.isdigit():
            option_index = int(user_answer) - 1
            if 0 <= option_index < len(question["options"]):
                user_answer = question["options"][option_index].lower()

        if user_answer == correct_answer:
            self.quiz_score += 1
            response = "✅ **Correct!** "
        else:
            response = f"❌ **Incorrect**. The answer is: {question['answer']}\n"

        self.current_question_index += 1

        if self.current_question_index < len(self.current_quiz):
            response += "\n" + self._format_question(self.current_question_index)
        else:
            response += self.end_quiz()

        return response

    def end_quiz(self) -> str:
        """Calculate and display final quiz results"""
        self.quiz_active = False
        duration = (datetime.now() - self.start_time).seconds
        accuracy = (self.quiz_score / len(self.current_quiz)) * 100 if self.current_quiz else 0

        result = (
            f"🎉 **Quiz Complete!**\n\n"
            f"📊 Score: {self.quiz_score}/{len(self.current_quiz)}\n"
            f"📈 Accuracy: {accuracy:.1f}%\n"
            f"⏱️ Time: {duration} seconds\n\n"
        )

        if accuracy >= 80:
            result += "🌟 Excellent work! You've mastered this topic."
        elif accuracy >= 50:
            result += "👍 Good job! Review the incorrect answers to improve."
        else:
            result += "📚 Keep practicing! Try studying the topic again."

        return result

    def handle_message(self, message: str) -> str:
        """Process user input and generate response"""
        msg = message.lower().strip()

        if self.quiz_active:
            return self.check_answer(message)

        if re.search(r"hi|hello|hey", msg):
            return (
                "👋 **Hello! I'm your Study Assistant.**\n\n"
                "I can:\n"
                "- Explain topics (try: 'explain AI')\n"
                "- Give definitions (try: 'define algorithm')\n"
                "- Provide quizzes (try: 'quiz me on Python')\n"
                "- Share study tips (try: 'give me a study tip')"
            )

        elif re.search(r"bye|exit|quit", msg):
            return "👋 Goodbye! Happy studying!"

        elif re.search(r"explain|what is|tell me about", msg):
            topic = re.sub(r"explain|what is|tell me about|[\?\.\!]", "", msg, flags=re.IGNORECASE).strip()
            return self.explain_topic(topic)

        elif re.search(r"define|what is a", msg):
            term = re.sub(r"define|what is a|[\?\.\!]", "", msg, flags=re.IGNORECASE).strip()
            return self.get_definition(term)

        elif re.search(r"quiz|test|questions", msg):
            topic_match = re.search(r"quiz (on|about) (.+)", msg, re.IGNORECASE)
            topic = topic_match.group(2) if topic_match else None
            return self.start_quiz(topic)

        elif re.search(r"tip|advice|how to study", msg):
            return self.get_study_tip()

        elif re.search(r"help|what can you do", msg):
            return (
                "📖 **I can help with:**\n"
                "- Explain academic topics\n"
                "- Define technical terms\n"
                "- Give quizzes with instant feedback\n"
                "- Share study strategies\n\n"
                "Try: 'Explain photosynthesis', 'Define OOP', or 'Quiz me on Python'"
            )

        else:
            return random.choice([
                "🤔 I'm not sure what you mean. Try asking me to explain a topic or give a quiz.",
                "📚 I specialize in study help. Try: 'Explain AI' or 'Quiz me on algebra'",
                "💡 Need help? Try: 'Define algorithm' or 'Give me a study tip'"
            ])

# ====== INTERACTIVE CHAT UI ======
bot = StudyBot()
chat_history = widgets.Output(layout={'border': '1px solid gray', 'max_height': '400px', 'overflow': 'auto'})
input_box = widgets.Text(
    placeholder="Type your message here...",
    layout=widgets.Layout(width="70%", margin="0 10px 0 0")
)
send_button = widgets.Button(
    description="Send",
    button_style="success",
    layout=widgets.Layout(width="20%")
)

def on_send_clicked(b):
    user_input = input_box.value.strip()
    if user_input:
        with chat_history:
            display(Markdown(f"**You:** {user_input}"))
            display(Markdown(f"**Bot:** {bot.handle_message(user_input)}"))
            display(Markdown("---"))
        input_box.value = ""

send_button.on_click(on_send_clicked)

def on_input_submit(sender):
    on_send_clicked(sender)

input_box.on_submit(on_input_submit)

# Display the chat interface
display(widgets.VBox([
    widgets.HTML("<h1 style='color: #3b5998;'>🧠 Enhanced Study Helper Chatbot</h1>"),
    widgets.HTML("<i>Try: 'Explain AI', 'Define algorithm', 'Quiz me on Python', or 'Give me a study tip'</i>"),
    chat_history,
    widgets.HBox([input_box, send_button])
]))

display(Markdown("✅ **Chatbot is ready!** Type your message above and click Send."))

✅ **Chatbot is ready!** Type your message above and click Send.